# Slippage Paper 1

# Modelling Transaction Costs when Trades May Be Crowded: A Bayesian Network Using Partially Observable Orders Imbalance
Marie Brière, Charles-Albert Lehalle, Tamara Nefedova, Amine Raboun

In [14]:
from IPython.display import Latex

## Summary

### Abstract

This summary presents three different models to estimate the implementation shortfall (IS) of trades. The models incorporate various factors such as daily volatility, price impact, trade size, and order imbalance.


### Mathematical Models

## Model 1: Implementation Shortfall

Model 1 captures the implementation shortfall as a function of daily volatility and normalized trade size impact:

$$
\text{IS}_k(m,d) = \alpha \psi_k(d) + \beta \sigma^{GK}_k(d) \sqrt{\frac{Q_k(m)}{\text{ADV}_k(d)}} + \epsilon_k(m,d)
$$

where:

$$
-\text{IS}_k(m,d) \text{ is the implementation shortfall for stock  k on day d  for meta-order  m .}
$$
$$
-\psi_k(d) \text{ is the daily volatility of stock  k  on day d.}
$$
$$
-\sigma^{GK}_k(d) \text{ is the price impact coefficient for stock k on day d.}
$$
$$
-Q_k(m)  \text{ is the size of meta-order m for stock k.}
$$
$$
- \text{ADV}_k(d) \text{ is the average daily volume of stock k on day d}
$$
$$
- \alpha, \beta \text{are the coefficients.}
$$
$$
-\epsilon_k(m,d) \text{ is the error term.}
$$


## Model 2: Implementation Shortfall with Order Imbalance

Model 2 incorporates order imbalance along with daily volatility:

$$
\text{IS}_k(m,d) = \alpha \psi_k(d) + \gamma \sigma^{GK}_k(d) s_k(m) \text{Imb}_k(m,d) + \epsilon_k(m,d)
$$

where:

$$
- \text{IS}_k(m,d) \text{ is the implementation shortfall for stock } k \text{ on day } d \text{ for meta-order } m.
$$
$$
- \psi_k(d) \text{ is the daily volatility of stock } k \text{ on day } d.
$$
$$
- \sigma^{GK}_k(d) \text{ is the price impact coefficient for stock } k \text{ on day } d.
$$
$$
- s_k(m) \text{ is the sign of the meta-order } m \text{ for stock } k.
$$
$$
- \text{Imb}_k(m,d) \text{ is the order imbalance for stock } k \text{ on day } d \text{ for meta-order } m.
$$
$$
- \alpha, \gamma \text{ are the coefficients.}
$$
$$
- \epsilon_k(m,d) \text{ is the error term.}
$$


# Model 3 combines components of Models 1 and 2:
$$
\text{IS}_k(m,d) = \alpha \psi_k(d) + \beta \sigma^{GK}_k(d) \sqrt{\frac{Q_k(m)}{\text{ADV}_k(d)}} + \gamma \sigma^{GK}_k(d) s_k(m) \text{Imb}_k(m,d) + \epsilon_k(m,d)
$$

$$
\text{This model integrates both the normalized trade size impact and order imbalance.}


In [19]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

np.random.seed(42)

# Number of data points
n = 1000

# Generate synthetic data
data = {
    'IS': np.random.normal(0, 1, n),
    'Volatility': np.random.normal(0.2, 0.05, n),
    'PriceImpactCoeff': np.random.normal(0.3, 0.1, n),
    'TradeSize': np.random.normal(1000, 250, n),
    'ADV': np.random.normal(500000, 100000, n),
    'OrderImbalance': np.random.normal(0.5, 0.2, n),
    'TradeSign': np.random.choice([-1, 1], n)
}

df = pd.DataFrame(data)

# Model 1: IS = alpha * Volatility + beta * PriceImpactCoeff * sqrt(TradeSize / ADV)
X1 = df[['Volatility']]
X1['Term'] = df['PriceImpactCoeff'] * np.sqrt(df['TradeSize'] / df['ADV'])
y = df['IS']

# Add constant term for intercept
X1 = sm.add_constant(X1)
model1 = sm.OLS(y, X1).fit()
print(model1.summary())

# Model 2: IS = alpha * Volatility + gamma * PriceImpactCoeff * TradeSign * OrderImbalance
X2 = df[['Volatility']]
X2['Term'] = df['PriceImpactCoeff'] * df['TradeSign'] * df['OrderImbalance']
y = df['IS']

# Add constant term for intercept
X2 = sm.add_constant(X2)
model2 = sm.OLS(y, X2).fit()
print(model2.summary())

# Model 3: IS = alpha * Volatility + beta * PriceImpactCoeff * sqrt(TradeSize / ADV) + gamma * PriceImpactCoeff * TradeSign * OrderImbalance
X3 = df[['Volatility']]
X3['Term1'] = df['PriceImpactCoeff'] * np.sqrt(df['TradeSize'] / df['ADV'])
X3['Term2'] = df['PriceImpactCoeff'] * df['TradeSign'] * df['OrderImbalance']
y = df['IS']

# Add constant term for intercept
X3 = sm.add_constant(X3)
model3 = sm.OLS(y, X3).fit()
print(model3.summary())


C:\Users\usre\AppData\Local\Temp\ipykernel_105660\2125079118.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X1['Term'] = df['PriceImpactCoeff'] * np.sqrt(df['TradeSize'] / df['ADV'])
C:\Users\usre\AppData\Local\Temp\ipykernel_105660\2125079118.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X2['Term'] = df['PriceImpactCoeff'] * df['TradeSign'] * df['OrderImbalance']
C:\Users\usre\AppData\Local\Temp\ipykernel_105660\2125079118.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy o

                            OLS Regression Results                            
Dep. Variable:                     IS   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.051
Date:                Wed, 26 Jun 2024   Prob (F-statistic):              0.350
Time:                        19:24:14   Log-Likelihood:                -1396.4
No. Observations:                1000   AIC:                             2799.
Df Residuals:                     997   BIC:                             2813.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1214      0.156      0.777      0.4